In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import csv
import shutil
import pandas as pd
import warnings
import os
import logging
import pyautogui


In [2]:
airlines = ["Alaska Airlines Inc. (AS)",
            "American Airlines Inc. (AA)",
            "Delta Airlines Inc. (DL)",
            "Frontier Airlines Inc. (F9)",
            "JetBlue Airways (B6)",
            "Southwest Airlines Co. (WN)",
            "Spirit Airlines (NK)",
            "United Airlines Inc. (UA)",
            "America West Airlines Inc. (HP)",
            "Atlantic Southeast Airlines (EV)",
            "Continental Airlines Inc. (CO)",
            "US Airways Inc. (US)",
            "SkyWest Airlines Inc. (OO)",
            "Republic Airline (YX)",
            "Endeavor Air Inc. (9E)",
            "American Eagle Airlines Inc. (MQ)",
            ]


In [3]:
# Path to the Chrome WebDriver executable
driverpath = "../maura/chromedriver.exe"
# Create a Service object with the specified path
service = Service(driverpath)

In [22]:
# Set the download directory
download_dir = "../rawdata/iahhist/"
os.makedirs(download_dir, exist_ok=True)

In [16]:
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
        "download.default_directory": download_dir,
        "download.prompt_for_download": True,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

In [34]:
absolute_path = r"E:\Dropbox\Dropbox\GRADRICE - DA\Projects\Project3\drafts\rawdata\iahhist"

In [25]:
# Pass the Service object to webdriver.Chrome()
driver = webdriver.Chrome(service=service,options=options)
url = 'https://www.transtats.bts.gov/ontime/departures.aspx'
driver.get(url)

In [26]:
# Selecting check historical statistics

driver.find_element("id","chkAllStatistics").click()
driver.find_element("id","cboAirport").click()
#filter airport ot houston 
driver.find_element("xpath","//*[contains(text(),'George Bush Intercontinental/Houston (IAH)')]").click()
#click allmonths, all days and all years
   #click all dates and all years 
driver.find_element("id", "chkAllMonths").click()
driver.find_element("id", "chkAllDays").click()
driver.find_element("id", "chkAllYears").click()

In [27]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [20]:
# Selecting the airlines
csv_names = []

In [42]:
# downloading all files

for index, airline in enumerate(airlines):
    
    try:
        # Filter airport to Houston
        time.sleep(10)
        element = driver.find_element("id", "cboAirline")
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        
        # Wait for the dropdown menu to be clickable
        wait = WebDriverWait(driver, 10)
        dropdown_menu = wait.until(EC.element_to_be_clickable((By.ID, "cboAirline")))
        
        # Click the dropdown menu
        dropdown_menu.click()
        
        # Click the airline option
        airline_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(),'" + airline + "')]")))
        airline_option.click()
        
        # Click submit button
        driver.find_element("id", "btnSubmit").click()
        # Wait for page to load
        time.sleep(5)

        # Export the CSV
        driver.find_element("id", "DL_CSV").click()
        # Wait for the download
        time.sleep(20)

        # Use pyautogui to automate keyboard inputs
        pyautogui.typewrite(["tab","tab","tab","tab","tab","tab"])  # tab to the file directory field
        pyautogui.press("enter") # Press Enter to confirm the path
        pyautogui.typewrite(absolute_path)
        pyautogui.press("enter") # Press Enter to confirm the path
        pyautogui.press("enter")  # Press Enter to save the file
            
    except Exception as e:
        logger.error(f"Error occurred for airline {airline}: {str(e)}")